In [86]:
import torch
from torch import Tensor
from torch.autograd import Variable
from torch import nn
from torch.nn import functional as F

from helpers import *

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [155]:
import dlc_bci as bci
train_input , train_target = bci.load(root = './data_bci')
print(str(type(train_input)), train_input.size()) 
print(str(type(train_target)), train_target.size())
test_input , test_target = bci.load(root = './data_bci', train = False)
print(str(type(test_input)), test_input.size()) 
print(str(type(test_target)), test_target.size())

<class 'torch.FloatTensor'> torch.Size([316, 28, 50])
<class 'torch.LongTensor'> torch.Size([316])
<class 'torch.FloatTensor'> torch.Size([100, 28, 50])
<class 'torch.LongTensor'> torch.Size([100])


In [2]:
import dlc_bci as bci
train_input , train_target = bci.load(root = './data_bci', one_khz = True)
print(str(type(train_input)), train_input.size()) 
print(str(type(train_target)), train_target.size())
test_input , test_target = bci.load(root = './data_bci', train = False, one_khz = True)
print(str(type(test_input)), test_input.size()) 
print(str(type(test_target)), test_target.size())

<class 'torch.FloatTensor'> torch.Size([316, 28, 500])
<class 'torch.LongTensor'> torch.Size([316])
<class 'torch.FloatTensor'> torch.Size([100, 28, 500])
<class 'torch.LongTensor'> torch.Size([100])


In [3]:
train_input[0]


  41.8000   44.8000   47.1000  ...    69.8000   72.6000   76.1000
 -10.3000   -5.9000   -3.3000  ...    12.6000   24.0000   26.5000
  38.1000   25.2000   46.0000  ...    45.1000   74.1000   64.8000
             ...                ⋱                ...             
   7.9000   11.2000   14.3000  ...    32.7000   43.4000   45.5000
  19.2000   33.6000   33.8000  ...    46.7000   53.7000   43.4000
  -0.4000   12.7000   12.0000  ...    30.7000   40.6000   33.1000
[torch.FloatTensor of size 28x50]

In [4]:
train_target


 0
 1
 0
 1
 0
 1
 0
 1
 0
 0
 1
 1
 0
 0
 1
 1
 1
 0
 0
 1
 0
 0
 1
 0
 1
 0
 0
 0
 0
 0
 0
 0
 1
 1
 1
 1
 1
 1
 0
 1
 0
 1
 0
 0
 1
 0
 1
 1
 0
 1
 1
 0
 0
 1
 0
 0
 1
 0
 1
 0
 1
 0
 0
 1
 0
 0
 0
 0
 0
 1
 0
 1
 0
 0
 0
 0
 0
 1
 1
 1
 0
 1
 0
 1
 1
 1
 1
 1
 1
 1
 1
 1
 0
 0
 0
 0
 1
 1
 1
 0
 1
 1
 1
 0
 1
 0
 1
 1
 1
 0
 1
 0
 1
 0
 0
 0
 1
 0
 0
 1
 1
 1
 1
 0
 1
 1
 0
 0
 1
 1
 1
 1
 1
 0
 0
 0
 0
 1
 1
 0
 1
 1
 1
 1
 1
 1
 1
 0
 0
 0
 1
 1
 0
 0
 1
 1
 0
 0
 0
 0
 0
 1
 1
 1
 1
 1
 1
 0
 1
 0
 1
 1
 0
 1
 0
 0
 1
 1
 1
 0
 0
 1
 0
 0
 0
 0
 0
 1
 1
 0
 0
 0
 0
 1
 0
 0
 0
 1
 0
 0
 1
 0
 0
 0
 1
 1
 1
 0
 0
 0
 1
 1
 1
 0
 0
 0
 1
 0
 1
 1
 1
 1
 1
 1
 1
 0
 1
 0
 0
 0
 1
 0
 1
 1
 1
 0
 1
 1
 0
 0
 0
 1
 1
 0
 1
 1
 0
 1
 1
 1
 1
 1
 0
 1
 0
 1
 0
 0
 0
 0
 0
 0
 1
 1
 1
 0
 1
 1
 1
 0
 1
 0
 0
 1
 1
 0
 0
 1
 0
 0
 1
 0
 0
 0
 1
 0
 0
 1
 1
 0
 0
 1
 1
 1
 0
 1
 0
 1
 1
 1
 0
 1
 1
 0
 1
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
[torch.LongTensor of size 316]

## Logistic Regression

In [5]:
from sklearn.linear_model import LogisticRegression

In [6]:
logistic_reg = LogisticRegression()

In [7]:
logistic_reg.fit(train_input.view(train_input.size(0),-1),train_target)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [8]:
pred = logistic_reg.predict(test_input.view(test_input.size(0),-1))

In [9]:
100*(1-(torch.LongTensor(pred) - test_target).abs().sum()/test_input.size(0))

74.0

## Neural Networks

#### Preprocessing the data

In [156]:
# normalize mean to 0
train_input.sub_(train_input.mean())
test_input.sub_(test_input.mean())

# normalize variance to 1
train_input.div_(train_input.std())
test_input.div_(test_input.std())

print('done')

done


In [100]:
#convert targets to one hot labels
train_target = convert_to_one_hot_labels(train_input, train_target)
test_target = convert_to_one_hot_labels(test_input, test_target)

### Basic CNN

In [172]:
class Basic_CNN(nn.Module):
    def __init__(self, nb_hidden=64):
        super(Basic_CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=(1,5))
        self.conv2 = nn.Conv2d(32, 64, kernel_size=(1,5))
        #self.dropout = nn.Dropout(0.5)
        self.fc1 = nn.Linear(640, nb_hidden)
        self.fc2 = nn.Linear(nb_hidden, 2)

    def forward(self, x):
        #print("======")
        #print(x.size())
        x = F.tanh(F.max_pool2d(self.conv1(x), kernel_size=3, stride=3))
        #print(x.size())
        x = F.tanh(F.max_pool2d(self.conv2(x), kernel_size=2, stride=2))
        #print(x.size())
        #x = self.dropout(x)
        x = F.tanh(self.fc1(x.view(-1, 640)))
        #print(x.size())
        x = self.fc2(x)
        #print("======")
        return x

In [180]:
class experimental_CNN(nn.Module):
    def __init__(self, nb_hidden=64):
        super(experimental_CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=(1,5))
        self.conv2 = nn.Conv2d(16, 32, kernel_size=(1,3))
        self.conv3 = nn.Conv2d(32, 32, kernel_size=(1,5))
        #self.dropout = nn.Dropout(0.5)
        self.fc1 = nn.Linear(896, nb_hidden)
        self.fc2 = nn.Linear(nb_hidden, 2)

    def forward(self, x):
        #print("======")
        #print(x.size())
        x = F.tanh(F.max_pool2d(self.conv1(x), kernel_size=(1,2), stride=(1,2)))
        #print(x.size())
        x = F.tanh(F.max_pool2d(self.conv2(x), kernel_size=(1,3), stride=(1,3)))
        #print(x.size())
        x = F.tanh(F.max_pool2d(self.conv3(x), kernel_size=(1,3), stride=(1,3)))
        #x = self.dropout(x)
        x = F.tanh(self.fc1(x.view(-1, 896)))
        #print(x.size())
        x = self.fc2(x)
        #print("======")
        return x

In [181]:
#cnn = Basic_CNN()
cnn = experimental_CNN()

In [182]:
train_model(cnn, Variable(train_input.view(-1, 1, 28, 50)), Variable(train_target), 4, learning_rate=0.01)

0 55.095009446144104
1 54.99918049573898
2 54.92152577638626
3 54.84894508123398
4 54.780639827251434
5 54.71426981687546
6 54.647876620292664
7 54.57976907491684
8 54.50793778896332
9 54.430730640888214
10 54.34650015830994
11 54.253846645355225
12 54.15158933401108
13 54.038151264190674
14 53.91265952587128
15 53.774734139442444
16 53.62448137998581
17 53.46393597126007
18 53.29462254047394
19 53.118749141693115
20 52.93729496002197
21 52.7488893866539
22 52.551105320453644
23 52.34062194824219
24 52.11320811510086
25 51.86375144124031
26 51.58703017234802
27 51.27950391173363
28 50.93730083107948
29 50.555081397295
30 50.122004598379135
31 49.63593265414238
32 49.08892187476158
33 48.47401976585388
34 47.784880608320236
35 47.01476362347603
36 46.1605309844017
37 45.216025829315186
38 44.177474707365036
39 43.04993197321892
40 41.85715167224407
41 40.62057623267174
42 39.359276711940765
43 38.090604677796364
44 36.839844703674316
45 35.61009614914656
46 34.402923196554184
47 33.2045

In [183]:
train_error_cnn = compute_nb_errors(cnn, Variable(train_input.view(-1, 1, 28, 50)), Variable(train_target), 4)
print(100*(1-(train_error_cnn/train_input.size(0))),'% training accuracy')

100.0 % training accuracy


In [184]:
test_error_cnn = compute_nb_errors(cnn, Variable(test_input.view(-1, 1, 28, 50)), Variable(test_target), 4)
print(100*(1-(test_error_cnn/test_input.size(0))),'% test accuracy')

78.0 % test accuracy


In [144]:
output = cnn(Variable(test_input.view(-1, 1, 28, 50)))
_, predicted_classes = torch.max(output.data, 1)
predicted_classes


 1
 0
 0
 0
 1
 0
 0
 0
 0
 0
 0
 0
 1
 1
 1
 0
 0
 1
 1
 0
 1
 1
 1
 1
 0
 1
 1
 1
 0
 0
 0
 0
 0
 1
 0
 0
 1
 1
 1
 1
 1
 1
 0
 0
 0
 1
 1
 0
 1
 1
 1
 1
 0
 0
 0
 1
 1
 0
 0
 1
 1
 0
 1
 1
 0
 0
 0
 1
 1
 1
 0
 1
 1
 1
 0
 0
 1
 0
 0
 1
 0
 1
 1
 0
 1
 1
 0
 1
 1
 0
 0
 0
 1
 0
 1
 1
 0
 0
 0
 0
[torch.LongTensor of size 100]

In [145]:
output

Variable containing:
-0.7196  0.6930
 0.6668 -0.6224
 1.0499 -1.0930
 0.4554 -0.4490
-1.0458  1.0564
 0.4398 -0.4385
 0.2946 -0.2884
 0.3804 -0.3914
 0.6147 -0.6129
 0.6399 -0.6136
 0.0827 -0.0386
 0.2206 -0.2163
-0.2460  0.1662
-0.8378  0.8327
-0.6485  0.6702
 0.2339 -0.2571
 0.5485 -0.5420
-0.3975  0.4288
-0.6710  0.6800
 1.3327 -1.3745
-0.3952  0.3567
-0.3144  0.3394
-0.1057  0.0654
-0.0207 -0.0128
 0.1883 -0.2306
-0.1562  0.1888
-0.6425  0.6204
-1.6709  1.6749
 0.8789 -0.8450
 0.1683 -0.1773
 0.2310 -0.2369
 1.4392 -1.4383
 0.4689 -0.4099
-1.2051  1.2111
 0.1250 -0.1663
 0.4937 -0.5129
-0.7417  0.7450
-0.3211  0.3298
-0.2980  0.3249
-0.3036  0.3132
-0.3558  0.3548
-0.9419  0.9164
 1.0998 -1.0973
 0.3867 -0.4003
 0.2432 -0.2202
-0.7651  0.7783
-0.4237  0.4156
 1.3667 -1.3871
-0.2305  0.2244
-0.4608  0.4342
-1.9480  1.9512
-0.7766  0.7487
 0.1940 -0.1450
 0.8935 -0.8784
 1.0315 -1.0242
-0.5146  0.4940
-0.9630  0.9805
 0.4162 -0.4086
 0.8827 -0.8671
-0.6418  0.6260
-1.2385  1.2261
 0.

### Basic MLP

In [146]:
MLP = nn.Sequential(
        nn.Linear(1400, 2000),
        nn.Tanh(),
        nn.Linear(2000, 1000),
        nn.Tanh(),
        nn.Linear(1000, 500),
        nn.Tanh(),
        nn.Linear(500, 2)
    )

In [147]:
train_model(MLP, Variable(train_input.view(train_input.size(0),-1)), Variable(train_target), 4)

0 84.22748374938965
1 78.6625856757164
2 76.88258475065231
3 75.6174096763134
4 74.4524522125721
5 73.27225422859192
6 72.0347093641758
7 70.72124433517456
8 69.32641085982323
9 67.85856041312218
10 66.33831188082695
11 64.79076832532883
12 63.238619327545166
13 61.699680134654045
14 60.18659369647503
15 58.70646731555462
16 57.262306079268456
17 55.854502737522125
18 54.481747180223465
19 53.14174821227789
20 51.83167317509651
21 50.54841583222151
22 49.28881733492017
23 48.049859169870615
24 46.82878374494612
25 45.62308695912361
26 44.43051684927195
27 43.24908941425383
28 42.07707139104605
29 40.9129038406536
30 39.75506218150258
31 38.60188074968755
32 37.451371217146516
33 36.3011418543756
34 35.14850836805999
35 33.991073086857796
36 32.828062711283565
37 31.661880346015096
38 30.496470361948013
39 29.326055778190494
40 28.11888513341546
41 26.809642999898642
42 25.283866177604068
43 27.8936515878886
44 25.90353285893798
45 24.137515405891463
46 26.614787702681497
47 19.51245481

In [152]:
train_error_mlp = compute_nb_errors(MLP, Variable(train_input.view(train_input.size(0),-1)), Variable(train_target), 4)
print(100*(1-(train_error_mlp/train_input.size(0))),'% training accuracy')

85.44303797468355 % training accuracy


In [154]:
test_error_mlp = compute_nb_errors(MLP, Variable(test_input.view(test_input.size(0),-1)), Variable(test_target), 4)
print(100*(1-(test_error_mlp/test_input.size(0))),'% test accuracy')

70.0 % test accuracy


In [ ]:
def find_parameters(model, step_sizes, nb_epochs):
    for eta in step_sizes:
        for epoch in nb_epochs:
            model = Basic_CNN()
            train_model(model)